# Auswerten eines Schach-Endspiels

## Imports, die für dieses Notebook benötigt werden

In [ ]:
import chess
from IPython.display import clear_output
from IPython.display import display
import json
from stockfish import Stockfish
import time

## Ausgangssituation

An dieser Stelle kann eine FEN eingegeben werden, welche die Figuren enthält, mit welcher die $S_n$ Sequenz aus der
ebenfalls einzugebenden Datei berechnet wurde.

Weiter muss der Pfad zu einer Stockfish-Installation angegeben werden.

In [ ]:
fen = "4k3/8/8/8/8/8/8/R3K3 w - - 0 1"
file = "S_n_seq_12_01.json"
STOCKFISH_PATH = "./stockfish_14.1/stockfish_14.1_win_x64_avx2.exe"
original_board = chess.Board(fen)

## Import der Daten

Die Daten wurden als FEN in der JSON-Datei Serialisiert.
Zum Initialisieren der Liste werden alle FENs gelesen und Board-Objekte erstellt.

In [ ]:
S_n_sequence_new = []
f = open(file, "r")
tmp = json.loads(f.read())
for item in tmp:
    tmp_list = []
    for board in item:
        tmp_list.append(chess.Board(board))
    S_n_sequence_new.append(tmp_list)
f.close()

## findBoardInSequence Hilfsfunktion

Diese Funktion durchsucht eine $S_n$ Sequenz nach dem ersten Vorkommen eines übergebenen Board-Objekts.

Funktions-Argumente:
* situation: Das Board (als Objekt), welches gefunden werden soll
* sequence: Die $S_n$ Sequenz, in welcher das Board gesucht wird

Ergebnis der Ausführung:
* Die Funktion hat zwei mögliche Rückgaben:
  * Ein Tupel mit $S_n$ Index (z.B. $S_3$) und Board-Index (z.B. 100).
    Dieses Tupel drückt aus, wo in der Sequenz das Board gefunden wurde.
  * Das Tupel (-1,-1). Dies drückt aus, dass das Board nicht gefunden wurde.

Nebeneffekte:
Die Funktion verändert keinen der übergebenen Parameter.

Algorithmus:
1. Über die $S_n$ Sequenz iterieren.
2. Über jedes Board in einem spezifischem $S_n$ iterieren.
3. Das Board mit dem _situation_ Objekt vergleichen.
    1. Wenn das Board übereinstimmt, die Indizes zurückgeben.
    2. Wenn das Board nicht übereinstimmt, weitersuchen.

In [ ]:
def findBoardInSequence(situation, sequence):
    board_str = (situation.turn, situation.__str__())

    for i in range(len(sequence)):
        for j in range(len(sequence[i])):
            item = sequence[i][j]
            if board_str == (item.turn, item.__str__()):
                return i, j
    return -1,-1

## Finden des ersten Boards in der $S_n$ Sequenz

Mithilfe der zuvor definierten _findBoardInSequence_ Funktion wird das *original_board* in der *S_n_sequence* gesucht.

In [ ]:
s_index = 0
board_index = 0

s_index, board_index = findBoardInSequence(original_board, S_n_sequence_new)

#Print search result
print("S" + str(s_index) + " - Board: " + str(board_index))

## Berechnen der Züge bis zum Spielende
Diese Berechnung nutzt die Stockfish-Engine zum Berechnen der Spielzüge für den schwarzen Spieler.

Voraussetzungen:
  * Die vorherige Zelle muss ausgeführt worden sein; Es müssen ein s_index und ein board_index gesetzt sein.

Ablauf:
  * Das aktuelle Board kopieren (Auswählen aus $S_n$ und kopieren)
  * Überprüfen ob Weiß oder Schwarz am Zug ist:
    * Wenn Weiß:
      * Alle legal moves ausprobieren, bis ein Spielzug gefunden wird, welcher das Board in einen Zustand aus $S_{n-1}$
      bringt.
    * Wenn Schwarz:
      * Stockfish einen Zug berechnen lassen.
      * Überprüfen in welcher $S_n$ Menge das Board sich nun befindet.
  * s_index und board_index aktualisieren
  * Wiederholen bis, das Board in $S_0$ ist (s_index = 0) oder kein Move gefunden wurde (s_index = -1)

In [ ]:
stockfish = Stockfish(STOCKFISH_PATH)

moves = []
moves = []
while s_index > 0:
    curr_board = S_n_sequence_new[s_index][board_index].copy()
    stockfish.set_fen_position(curr_board.fen())
    if curr_board.turn:
        print("---White:---")
        print("Starting in S" + str(s_index))
        placement_dict = {}
        for move in curr_board.legal_moves:
            curr_board.push(move)
            _tmp, board_index_tmp = findBoardInSequence(curr_board, [S_n_sequence_new[s_index - 1]])
            s_index_tmp = s_index - 1
            if board_index_tmp != -1:
                print("    Move: " + str(move))
                print("    S" + str(s_index_tmp) + " - Board: " + str(board_index_tmp))

                moves.append(move)
                curr_board.pop()
                s_index = s_index_tmp
                board_index = board_index_tmp
                break
            curr_board.pop()
        print("Ended in S" + str(s_index))

    else:
        print("---Black:---")
        print("Starting in S" + str(s_index))
        #move_list = list(curr_board.legal_moves)
        #move_index = random.randint(0, len(move_list) - 1)
        move = chess.Move.from_uci(stockfish.get_best_move())
        curr_board.push(move)
        s_index, board_index = findBoardInSequence(curr_board, S_n_sequence_new[:s_index])
        print("    Move: " + str(move))
        print("    S" + str(s_index) + " - Board: " + str(board_index))
        if s_index == -1:
            print("No solution")
            break
        moves.append(move)
        print("Ended in S" + str(s_index))


print("Found " + str(len(moves)) + " moves to win")

## Ergebnis mit Stockfish Ergebnis vergleichen
Als Benchmark für die Berechnung werden alle Spielzüge für sowohl Weiß als auch Schwarz mit der Stockfish Enginge berechnet.
Anschließend kann verglichen werden, wie viele Züge beide Ansätze benötigt haben.

In [ ]:
compare_board = chess.Board()
compare_board.set_fen(fen)
compare_moves = []

while not compare_board.is_game_over():
    stockfish.set_fen_position(compare_board.fen())
    next_move = chess.Move.from_uci(stockfish.get_best_move())
    compare_board.push(next_move)
    compare_moves.append(next_move)

print("Stockfish ended the Game in " + str(len(compare_moves)) + " turns.")

## Anzeigen des Ergebnisses
Ergebnisse anzeigen, in dem mit 2 Sekunden Verzögerung alle Moves durchgeführt werden.

In [ ]:
presentation_board = original_board.copy()
display(presentation_board)
time.sleep(2)
for move in moves:
    presentation_board.push(move)
    clear_output(wait=True)
    display(presentation_board)
    time.sleep(2)